# Identify GTFS shape overlap issues

Known issue from RT speedmaps: vehicle positions are "snapped" to shapes in wrong order when a route starts and ends in the same place. Calculated speeds are implausible and trips are dropped.

Goal: Identify how many shapes have this issue.

In [1]:
import calitp
import geopandas as gpd
import numpy as np
import pandas as pd
import shared_utils
import datetime as dt
from siuba import * 

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

pd.set_option("display.max_columns", None)

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# set testing parameters - test on known operator w/ issue (KART)
analysis_operator=148
analysis_dt= dt.date(2022, 6, 1)

In [3]:
help(shared_utils.gtfs_utils.get_route_shapes)

Help on function get_route_shapes in module shared_utils.gtfs_utils:

get_route_shapes(selected_date: str | datetime.date, itp_id_list: list[int] = None, get_df: bool = True, crs: str = 'EPSG:4326', trip_df: siuba.sql.verbs.LazyTbl | pandas.core.frame.DataFrame = None, custom_filtering: dict = None) -> geopandas.geodataframe.GeoDataFrame
    Return a subset of geography_utils.make_routes_gdf()
    to only have the `shape_id` values present on a selected day.
    
    geography_utils.make_routes_gdf() only selects based on calitp_extracted_at
    and calitp_deleted_at date range.
    
    Allow a pre-existing trips table to be supplied.
    If not, run a fresh trips query.
    
    Custom_filtering doesn't filter in the query (which relies on trips query),
    but can filter out after it's a gpd.GeoDataFrame



In [4]:
gdf = (shared_utils.gtfs_utils.get_route_shapes(selected_date = analysis_dt,itp_id_list = [analysis_operator]))

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 5000000000. 16087252992 or higher required.

Location: us-west2
Job ID: d1305c0e-c36b-4ece-9307-8f0b3fbf9b32

[SQL: SELECT DISTINCT `anon_1`.`calitp_itp_id`, `anon_1`.`calitp_url_number`, `anon_1`.`shape_id` 
FROM (SELECT `anon_2`.`feed_key` AS `feed_key`, `anon_2`.`trip_key` AS `trip_key`, `anon_2`.`trip_id` AS `trip_id`, `anon_2`.`route_id` AS `route_id`, `anon_2`.`calitp_itp_id` AS `calitp_itp_id`, `anon_2`.`calitp_url_number` AS `calitp_url_number`, `anon_2`.`service_id` AS `service_id`, `anon_2`.`service_date` AS `service_date`, `anon_2`.`service_indicator` AS `service_indicator`, `anon_2`.`service_start_date` AS `service_start_date`, `anon_2`.`service_end_date` AS `service_end_date`, `anon_2`.`service_inclusion` AS `service_inclusion`, `anon_2`.`service_exclusion` AS `service_exclusion`, `anon_2`.`is_in_service` AS `is_in_service`, `anon_2`.`n_stops` AS `n_stops`, `ano